In [ ]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path

# required to enable non-blocking interaction:
# %gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
# from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import session_context_to_relative_path
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.result_context import IdentifyingContext

from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import generate_batch_single_session_scripts
global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]
## Get the context:basedir dict for the relevant contexts:
output_session_basedir_dict = {k:v for k, v in KDibaOldDataSessionFormatRegisteredClass.build_session_basedirs_dict(global_data_root_parent_path).items() if k in included_session_contexts}
# output_session_basedir_dict


In [ ]:
included_session_contexts, output_python_scripts, output_slurm_scripts = generate_batch_single_session_scripts(global_data_root_parent_path, session_batch_basedirs=output_session_basedir_dict, included_session_contexts=included_session_contexts, output_directory=Path('../output/gen_scripts/').resolve(),  use_separate_run_directories=True, create_slurm_scripts=False,
	should_force_reload_all=False, should_perform_figure_generation_to_file=False)
output_python_scripts

# List Selector:

In [ ]:
from pyphocorehelpers.gui.Jupyter.simple_widgets import build_dropdown_selection_widget

active_slected_context = included_session_contexts[0]

def selected_context_changed_callback(context_str, context_data):
    global active_slected_context
    print(f"You selected {context_str}:{context_data}")
    active_slected_context = context_data # update the selected context

context_selection_dropdown = build_dropdown_selection_widget(included_session_contexts, on_user_update_item_selection=selected_context_changed_callback)
context_selection_dropdown

In [ ]:
active_slected_context

In [ ]:

def selected_context_changed_callback(change):
    selected_item = change['new']
    print(f"You selected {selected_item}")
    # Run your function or cell code here

# item_list = ['Item 1', 'Item 2', 'Item 3']
item_list = included_session_contexts.copy()
context_selection_dropdown = widgets.Dropdown(options=item_list, description='Select Context:', layout=widgets.Layout(width='auto'))
context_selection_dropdown.style = {'description_width': 'initial'} # Increase dropdown width with CSS
context_selection_dropdown.observe(selected_context_changed_callback, 'value')
display(context_selection_dropdown)


# Tree selector:

In [ ]:
from pyphocorehelpers.gui.Jupyter.TreeWidget import JupyterTreeWidget

jupyter_tree_widget = JupyterTreeWidget(included_session_contexts=included_session_contexts)
# jupyter_tree_widget

In [ ]:
from collections import defaultdict

def construct_tree(lst, keys):
    tree = defaultdict(list)
    if len(keys) == 1:
        for d in lst:
            tree[d[keys[0]]].append(d)
        return dict(tree)
    for d in lst:
        tree[d[keys[0]]].append(d)
    for k, v in tree.items():
        tree[k] = construct_tree(v, keys[1:])
    return dict(tree)


included_session_context_dict_tree = [ctxt.to_dict() for ctxt in included_session_contexts]

included_session_context_dict_tree

keys = ['format_name', 'animal', 'exper_name', 'session_name']
dict_list = included_session_context_dict_tree.copy()
# ... (rest of your data)]
tree = construct_tree(dict_list, keys)
tree

# Raw ipywidgets version using accordion

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def create_tree_widget(tree, level=0):
    if isinstance(tree, dict):
        acc = widgets.Accordion(children=[])
        for key, value in tree.items():
            child_widget = create_tree_widget(value, level+1)
            acc.children += (child_widget,)
            acc.set_title(len(acc.children)-1, str(key))
        return acc
    elif isinstance(tree, list):
        return widgets.VBox([widgets.Label(str(item)) for item in tree])
    else:
        return widgets.Label(str(tree))

# Your tree variable
# tree = ...

# Create widget tree from data
tree_widget = create_tree_widget(tree)

# Display the widget tree
display(tree_widget)

In [ ]:
selected_leaf = None

def on_leaf_click(b):
    global selected_leaf
    if selected_leaf:
        selected_leaf.button_style = ''
    b.button_style = 'success'
    selected_leaf = b

def create_tree_widget(tree, level=0):
    global selected_leaf
    if isinstance(tree, dict):
        acc = widgets.Accordion(children=[])
        acc.selected_index = list(range(len(tree)))  # Expand all
        for key, value in tree.items():
            child_widget = create_tree_widget(value, level+1)
            acc.children += (child_widget,)
            acc.set_title(len(acc.children)-1, str(key))
        return acc
    elif isinstance(tree, list):
        buttons = []
        for item in tree:
            button = widgets.Button(description=str(item['session_name']), layout=widgets.Layout(width='auto'))
            button.on_click(on_leaf_click)
            buttons.append(button)
        return widgets.VBox(buttons)
    else:
        return widgets.Label(str(tree))
    

# Create widget tree from data
tree_widget = create_tree_widget(tree)

# Display the widget tree
display(tree_widget)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

selected_leaf = None

def toggle_accordion(b):
    acc = b.accordion_reference
    if acc.selected_index is None:
        acc.selected_index = list(range(len(acc.children)))
    else:
        acc.selected_index = None

def on_leaf_click(b):
    global selected_leaf
    if selected_leaf:
        selected_leaf.button_style = ''
    b.button_style = 'success'
    selected_leaf = b

def create_tree_widget(tree, level=0):
    global selected_leaf
    if isinstance(tree, dict):
        acc = widgets.Accordion(children=[])
        toggle_button = widgets.Button(description="Toggle All")
        toggle_button.accordion_reference = acc
        toggle_button.on_click(toggle_accordion)
        
        for key, value in tree.items():
            child_widget = create_tree_widget(value, level+1)
            acc.children += (child_widget,)
            acc.set_title(len(acc.children)-1, str(key))
        return widgets.VBox([toggle_button, acc])
    elif isinstance(tree, list):
        buttons = []
        for item in tree:
            button = widgets.Button(description=str(item['session_name']), layout=widgets.Layout(width='auto'))
            button.on_click(on_leaf_click)
            buttons.append(button)
        return widgets.VBox(buttons)
    else:
        return widgets.Label(str(tree))

# Your tree variable
# tree = ...

# Create widget tree from data
tree_widget = create_tree_widget(tree)

# Display the widget tree
display(tree_widget)


# ipytree version:

In [ ]:


# def on_node_selected(change):
#     if change['new']:  # Node selected
#         print(f"Selected node: {change['owner'].name}")

# def build_tree(node, value, max_depth=20, depth=0):
#     """ 
#         import ipytree as ipyt
#         from IPython.display import display
#         from pyphocorehelpers.gui.Jupyter.TreeWidget import build_tree


#         # Assume curr_computations_results is the root of the data structure you want to explore
#         root_data = a_sess_config.preprocessing_parameters
#         root_node = ipyt.Node('sess.config')
#         build_tree(root_node, root_data)

#         tree = ipyt.Tree()
#         tree.add_node(root_node)
#         display(tree)
#     """
#     if depth > max_depth:
#         return

#     if isinstance(value, dict):
#         for key, child_value in value.items():
#             child_node = ipyt.Node(key)
#             node.add_node(child_node)
#             build_tree(child_node, child_value, max_depth=max_depth, depth=depth+1)
#     else:
#         ## add leaf node:
#         # node.add_node(ipyt.Node(str(value)))
#         leaf_node = ipyt.Node(str(value))
#         leaf_node.observe(on_node_selected, 'selected')
#         node.add_node(leaf_node)



# Assume curr_computations_results is the root of the data structure you want to explore
# root_data = a_sess_config.preprocessing_parameters
# root_node = ipyt.Node('sess.config')

root_data = tree
root_node = ipyt.Node('tree')
build_tree(root_node, root_data)

tree = ipyt.Tree()
tree.add_node(root_node)
display(tree)


In [ ]:
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.


# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # DONE, might be the BEST SESSION, good example session with lots of place cells, clean replays, and clear bar graphs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD, 2023-07-14, unsure why still.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE, Added replay selections. Very "jumpy" between the starts and ends of the track.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE, Added replay selections. A TON of putative replays in general, most bad, but some good.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE, added replay selections. Very few (like 12) replays each.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE, okay replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE, very few replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE, one replay each (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE, good. Many good pfs, many good replays. Noticed very strange jumping off the track in the 3D behavior/spikes viewer. Is there something wrong with this session?
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE, good?, replays selected, few
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE, very few replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE, replays selected, quite a few replays but few are very good.

basedir: Path = output_session_basedir_dict[curr_context]
print(f'basedir: {str(basedir)}')

In [ ]:

# # Read if possible:
# saving_mode = PipelineSavingScheme.SKIP_SAVING
# force_reload = False

# # # # # # Force write:
# # saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# # saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# # force_reload = True

## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.




# 2023-09-21 - `dill` Checking

In [ ]:
# dill.errors(obj, depth=0, exact=False, safe=False) # get errors for objects that fail to pickle
# dill.pickles(obj, exact=False, safe=False, **kwds) # Quick check if object pickles with dill.
# dill.get_objgraph

def dump_session(file_path):
  """Pickle the current python session to be used in the worker.

  Note: Due to the inconsistency in the first dump of dill dump_session we
  create and load the dump twice to have consistent results in the worker and
  the running session. Check: https://github.com/uqfoundation/dill/issues/195
  """
  dill.dump_session(file_path)
  dill.load_session(file_path)
  return dill.dump_session(file_path)

test_session_file = Path('../output/test_pickled_session_2023-09-21.pkl').resolve()
print(f'saving session to {test_session_file}')
dump_session(test_session_file)

# dill.detect.trace(True)
# dill.dump_session(test_session_file) # PicklingError: Can't pickle : it's not the same object as builtins.input
# PicklingError: Can't pickle .NewCol'>: it's not found as tables.description.Col._subclass_from_prefix..NewCol

# with detect.trace('output/dill_trace_2023-09-21.log', mode='w') as log:
#     # log("> D = %r", D)
#     dill.dump_session(test_session_file)


In [ ]:

dill.detect.trace(False)
# dump_session(test_session_file)


In [ ]:
 
dill.dump_session()